In [43]:
import Base: @kwdef
using Printf
using Random
using LaTeXStrings
using Transducers
using ProgressMeter
using CSV
using DataFrames
import Statistics
import Distances
import Distributions

In [2]:
using Plots
using StatsPlots
using ImageShow

# Data

In [11]:
data = CSV.read("data.csv", DataFrame)

Row,dist,2G_Power,3G_Power,4G_Power
,Int64,Int64,Int64,Int64
1,155,-63,-81,-81
2,152,-59,-83,-94
3,158,-65,-78,-89


In [17]:
height_measurement = 17#m
height_antenna = 38#m
data[:,"true_dist"] = data[:,"dist"] |>
  Map() do dist_antenna
    Distances.euclidean((0, height_measurement), (dist_antenna, height_antenna))
  end |> collect
data

Row,dist,2G_Power,3G_Power,4G_Power,true_dist
,Int64,Int64,Int64,Int64,Float64
1,155,-63,-81,-81,156.416
2,152,-59,-83,-94,153.444
3,158,-65,-78,-89,159.389


# Task

## 2G

Формула - *COST–Hata–Model*

$$
PL(d) = 46.3 + 33.9 \cdot \lg(f_C) − 13.82 \cdot \lg(h_{te}) − a(h_{re}) 
      + (44.9 − 6.55 \cdot \lg(h_{te})) \cdot \lg(d) + C_m \\

a(h_{re}) = 3.2 \cdot (\lg(11.75 \cdot h_{re}))^2 − 4.97
$$

Параметры:
 - $h_{te}$ -- эффективная высота базовой антенны в метрах (от $30$ до $200 м$)
 - $h_{re}$ -- эффективная высота мобильной антенны в метрах (от $1$ до $10 м$),
 - $f_C$ -- частота сигнала в МГц от $150$ до $1500 МГц$
 - $d$ -- расстояние от передатчика до приемника, $км$
 - $a(h_{re})$ -- корректирующий фактор для эффективной высоты мобильной антенны,
   который является функцией величины зоны обслуживания. Используется версия 
   формулы для крупных городов (1.12) в методичке.
 - $C_m$ = $3 дБ$ для крупных городо

In [28]:
𝑎(h_re) = 3.2 * log10(11.75 * h_re)^2 - 4.97
PL_2G(f_C, h_te, h_re, C_m, d) = 46.3 + 33.9 * log10(f_C) - 13.82 * log10(h_te) - 
    𝑎(h_re) + (44.9 - 6.55 * log10(h_te)) * log10(d) + C_m

PL_2G (generic function with 1 method)

In [32]:
f_C = 960; h_te = 30; h_re = 10; C_m = 3;
data[:, "2G_Attenuation"] = data[:,"true_dist"] |>
    Map() do dist
        db = PL_2G(f_C, h_te, h_re, C_m, dist / 1000.0)
        28 - db
    end |> collect
data

Row,dist,2G_Power,3G_Power,4G_Power,true_dist,2G_Attenuation
,Int64,Int64,Int64,Int64,Float64,Float64
1,155,-63,-81,-81,156.416,-64.8617
2,152,-59,-83,-94,153.444,-64.5682
3,158,-65,-78,-89,159.389,-65.1498


In [48]:
σ_2G = 12
data[:, "2G_prob"] = data[:,"2G_Power"] |>
    Map() do power
        dist = Distributions.LogNormal(power, σ_2G)
        Distributions.ccdf(dist, -100)
    end |> collect
data

Row,dist,2G_Power,3G_Power,4G_Power,true_dist,2G_Attenuation,3G_Attenuation,4G_Attenuation,2G_prob
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64
1,155,-63,-81,-81,156.416,-64.8617,-74.8284,-72.5996,1.0
2,152,-59,-83,-94,153.444,-64.5682,-74.5785,-72.2739,1.0
3,158,-65,-78,-89,159.389,-65.1498,-75.0738,-72.9192,1.0


## 3G

$$
PL(d) = 37 + 30 \cdot \lg(d) + 18.3 \cdot 𝑛^{(\frac{𝑛+2}{𝑛+1} − 0.46)}
$$

Параметры:
 - $d$ – расстояние между передатчиком и приемником (м),
 - $n$ – количество этажей на пути.

In [33]:
PL_3G(n, d) = 37 + 30 * log10(d) + 18.3 * n^((n + 2) / (n + 1) - 0.46)

PL_3G (generic function with 1 method)

In [35]:
n = 0;
data[:, "3G_Attenuation"] = data[:,"true_dist"] |>
    Map() do dist
        db = PL_3G(n, dist)
        28 - db
    end |> collect
data

Row,dist,2G_Power,3G_Power,4G_Power,true_dist,2G_Attenuation,3G_Attenuation
,Int64,Int64,Int64,Int64,Float64,Float64,Float64
1,155,-63,-81,-81,156.416,-64.8617,-74.8284
2,152,-59,-83,-94,153.444,-64.5682,-74.5785
3,158,-65,-78,-89,159.389,-65.1498,-75.0738


In [49]:
σ_3G = 12
data[:, "3G_prob"] = data[:,"3G_Power"] |>
    Map() do power
        dist = Distributions.LogNormal(power, σ_3G)
        Distributions.ccdf(dist, -100)
    end |> collect
data

Row,dist,2G_Power,3G_Power,4G_Power,true_dist,2G_Attenuation,3G_Attenuation,4G_Attenuation,2G_prob,3G_prob
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64
1,155,-63,-81,-81,156.416,-64.8617,-74.8284,-72.5996,1.0,1.0
2,152,-59,-83,-94,153.444,-64.5682,-74.5785,-72.2739,1.0,1.0
3,158,-65,-78,-89,159.389,-65.1498,-75.0738,-72.9192,1.0,1.0


## 4G

$$
PL_{3D-UMa-NLOS} = 161.04 – 7.1 \lg(W) + 7.5 \lg(h) – (24.37 – 3.7 (\frac{h}{h_{BS}})^2) \\
    \lg(h_{BS}) + (43.42 – 3.1 \lg(h_{BS})) (\lg(d_{3D}) - 3) + 20 \lg(f_C) –  \\
    (3.2 (\lg(17.625))^2 - 4.97) – 0.6(h_{UT} - 1.5)
$$

Параметры:
 - $f_C$ – частота сигнала в ГГц
 - $h$ – средняя высота зданий в диапазоне $5 м < h < 50 м$, типовое значение $h = 20 м$,
 - $W$ – ширина улицы в диапазоне $5 м < W < 50 м$, типовое значение $W = 20 м$
 - $h_{BS}$ и $h_{UT}$ – действительные высоты антенн; типовое значение 
   $h_{BS} = 25 м$ и $10 м < h_{BS} < 150 м$,

In [37]:
PL_4G(W, h, hBS, d3D, fc, hUT) = 161.04 - 7.1 * log10(W) + 7.5 * log10(h) - 
    (24.37 - 3.7* (h / hBS)^2) * log10(hBS) + (43.42 - 3.1 * log10(hBS)) * 
    (log10(d3D) - 3) + 20 * log10(fc) - (3.2 * log10(17.625)^2 - 4.97) -
    0.6 * (hUT - 1.5)

PL_4G (generic function with 1 method)

In [41]:
W = 20; h = 25; hBS = 25; fc = 2.6; hUT = 17.5
data[:, "4G_Attenuation"] = data[:,"true_dist"] |>
    Map() do dist
        db = PL_4G(W, h, hBS, dist, fc, hUT)
        28 - db
    end |> collect
data

Row,dist,2G_Power,3G_Power,4G_Power,true_dist,2G_Attenuation,3G_Attenuation,4G_Attenuation
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64
1,155,-63,-81,-81,156.416,-64.8617,-74.8284,-72.5996
2,152,-59,-83,-94,153.444,-64.5682,-74.5785,-72.2739
3,158,-65,-78,-89,159.389,-65.1498,-75.0738,-72.9192


In [50]:
σ_4G = 6
data[:, "4G_prob"] = data[:,"4G_Power"] |>
    Map() do power
        dist = Distributions.LogNormal(power, σ_4G)
        Distributions.ccdf(dist, -100)
    end |> collect
data

Row,dist,2G_Power,3G_Power,4G_Power,true_dist,2G_Attenuation,3G_Attenuation,4G_Attenuation,2G_prob,3G_prob,4G_prob
,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,155,-63,-81,-81,156.416,-64.8617,-74.8284,-72.5996,1.0,1.0,1.0
2,152,-59,-83,-94,153.444,-64.5682,-74.5785,-72.2739,1.0,1.0,1.0
3,158,-65,-78,-89,159.389,-65.1498,-75.0738,-72.9192,1.0,1.0,1.0


## Wi-Fi